In [40]:
import pandas as pd
import re
import csv
from IPython.display import clear_output

names = ['people_id','activity_id','date','activity_category','char_1','char_2','char_3','char_4','char_5','char_6','char_7','char_8','char_9','char_10','outcome']
pl_names = ['people_id','char_1','group_1','char_2','date','char_3','char_4','char_5','char_6','char_7','char_8','char_9','char_10','char_11','char_12','char_13','char_14','char_15','char_16','char_17','char_18','char_19','char_20','char_21','char_22','char_23','char_24','char_25','char_26','char_27','char_28','char_29','char_30','char_31','char_32','char_33','char_34','char_35','char_36','char_37','char_38']

In [41]:
def convert2int(v):
    return '1' if 'True' == v else '0' if 'False' == v else None if not v else re.search('\d+$', v).group(0)

In [42]:
act_train = pd.read_csv('act_train.csv', names=names, converters={x: convert2int for x in names[3:-1]}, dtype=str)
act_test = pd.read_csv('act_test.csv', names=names[:-1], converters={x: convert2int for x in names[3:-1]}, dtype=str)
people = pd.read_csv('people.csv', names=pl_names, converters={x: convert2int for x in pl_names[1:4]+pl_names[5:-1]}, dtype=str)
# print act_train.head()
# print act_test.head()
# print people.head()
# clear_output()

In [43]:
#value count the appearance of people in activity table 
cnt = act_train.people_id.value_counts()
people_act_cnt = pd.DataFrame({'people_id': cnt.index, 'ppl_act_cnt': cnt.values})
people_act_cnt.head()

,people_id,ppl_act_cnt
0,ppl_294918,55103
1,ppl_370270,53668
2,ppl_105739,45936
3,ppl_54699,23969
4,ppl_64887,7052


In [44]:
#add above people activity count as a new feature add to train/test data
people = pd.merge(people, people_act_cnt, how='outer')
people.fillna(1, inplace=True)
people.tail()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38,ppl_act_cnt
189113,ppl_99987,1,8600,1,2022-04-02,4,6,4,3,11,...,0,0,1,0,1,0,1,1,89,1
189114,ppl_9999,2,17304,2,2023-02-23,6,2,8,3,11,...,0,0,0,0,0,0,0,0,0,2
189115,ppl_99992,2,17304,2,2020-06-25,5,5,3,4,16,...,0,0,0,0,0,0,0,0,0,2
189116,ppl_99994,2,17764,3,2023-01-06,2,7,2,1,2,...,1,1,1,0,1,1,1,1,95,46
189117,ppl_99997,2,17304,2,2022-03-12,40,25,9,3,8,...,0,0,0,0,0,0,0,0,36,1


In [45]:
#train dataset
train_data = pd.merge(act_train, people, on=['people_id'])
label = train_data['outcome']
# removing id columns, date columns and outcome
feats = pd.concat([train_data.ix[:, 3:14], train_data.ix[:, 15:18], train_data.ix[:, 19:]], axis=1)
train_data_svm = pd.concat([label, feats], axis=1) 
feat_size = feats.shape[1]
print 'train_data_svm shape: (%d,%d)' % train_data_svm.shape
print 'train_data_svm columns:  %s' % ','.join(train_data_svm.columns.values)

train_data_svm shape: (2197291,52)
train_data_svm columns:  outcome,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,char_10_x,char_1_y,group_1,char_2_y,char_3_y,char_4_y,char_5_y,char_6_y,char_7_y,char_8_y,char_9_y,char_10_y,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38,ppl_act_cnt


In [46]:
# write svm format train data to file
with open('train_v2_svm.txt', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    rows = [' '.join([str(x[0])] + [str(i)+':'+str(x[i+1]) for i in range(feat_size) if x[i+1]]) for x in train_data_svm.values]
    for r in rows:
        writer.writerow([r.strip()])

### Below is for test dataset

In [47]:
#test dataset
test_data = pd.merge(act_test, people, on=['people_id'])
test_feats = pd.concat([test_data.ix[:, 1], test_data.ix[:, 3:17], test_data.ix[:, 18:]], axis=1)
print 'test data colum_names: %s' % ','.join(test_feats.columns.values)
test_feat_size = test_feats.shape[1]-1
print 'test data feature size: %d' % test_feat_size

test data colum_names: activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,char_10_x,char_1_y,group_1,char_2_y,char_3_y,char_4_y,char_5_y,char_6_y,char_7_y,char_8_y,char_9_y,char_10_y,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38,ppl_act_cnt
test data feature size: 51


In [48]:
# write svm format test data to file: feature data vs. activity id list
with open('test_v2_svm.txt', 'w') as ft, open('test_v2_svm_map.txt', 'w') as fm:
    twriter = csv.writer(ft, delimiter='\t')
    mwriter = csv.writer(fm, delimiter='\t')
    ids = test_feats.ix[:, 0].values
    idx = test_feats.index.values

    rows = [(str(x) + ',' + z, str(x) + ' ' + ' '.join([str(i)+':'+str(y[i]) for i in range(test_feat_size) if y[i]]))
            for x, z, y in zip(idx, ids, (test_feats.ix[:, 1:]).values)]

    for (x, y) in rows:
        twriter.writerow([y.strip()])
        mwriter.writerow([x.strip()])